### 1 - Import test data

In [4]:
!pip install yfinance

import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2024-9-20", end="2024-10-20", interval='15m')
dataF.iloc[:,:]
#dataF.Open.iloc

Defaulting to user installation because normal site-packages is not writeable


[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X
Datetime,,,,,,
2024-09-20 00:00:00+00:00,1.116196,1.116196,1.116321,1.116196,1.116321,0
2024-09-20 00:15:00+00:00,1.116196,1.116196,1.116321,1.116196,1.116321,0
2024-09-20 00:30:00+00:00,1.116196,1.116196,1.116321,1.116196,1.116321,0
2024-09-20 00:45:00+00:00,1.116321,1.116321,1.116321,1.116196,1.116321,0
2024-09-20 01:00:00+00:00,1.116196,1.116196,1.116445,1.116071,1.116321,0
...,...,...,...,...,...,...
2024-10-18 21:15:00+00:00,1.087193,1.087193,1.087193,1.086957,1.086957,0
2024-10-18 21:30:00+00:00,1.087075,1.087075,1.087311,1.086957,1.087193,0


### 2 - Define your signal function

In [17]:
def signal_generator(df):
    
    open = df["Open"].iloc[-1].iloc[0]
    close = df["Close"].iloc[-1].iloc[0]
    previous_open = df["Open"].iloc[-2].iloc[0]
    previous_close = df["Close"].iloc[-2].iloc[0]
    
    # Bearish Pattern
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

In [19]:
dataF.signal.value_counts()
# dataF.iloc[:, :]

Price,Adj Close,Close,High,Low,Open,Volume,signal
Ticker,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,
Datetime,,,,,,,
2024-09-20 00:00:00+00:00,1.116196,1.116196,1.116321,1.116196,1.116321,0,0
2024-09-20 00:15:00+00:00,1.116196,1.116196,1.116321,1.116196,1.116321,0,0
2024-09-20 00:30:00+00:00,1.116196,1.116196,1.116321,1.116196,1.116321,0,0
2024-09-20 00:45:00+00:00,1.116321,1.116321,1.116321,1.116196,1.116321,0,0
2024-09-20 01:00:00+00:00,1.116196,1.116196,1.116445,1.116071,1.116321,0,0
...,...,...,...,...,...,...,...
2024-10-18 21:15:00+00:00,1.087193,1.087193,1.087193,1.086957,1.086957,0,0
2024-10-18 21:30:00+00:00,1.087075,1.087075,1.087311,1.086957,1.087193,0,0


### 3 - Connect to the market and execute trades

In [21]:
!pip install apscheduler oandapyV20 oanda_candles

from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

2009.80s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 KB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.1/248.1 KB 5.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 KB 4.0 MB/s eta 0:00:00
  Created wheel for oandapyV20: filename=oandapyV20-0.6.3-py3-none-any.whl size=69759 sha256=c680a7b7ef2e61d8e9371f7cab2a6c5a2f9ae3e1f91068e90f94f2e0b71583d3
  Stored in directory: /home/peter/.cache/pip/wheels/15/56/f0/9bc4a64eebce14cb8a214b1818143a5010ef80786836dbc6c1
Successfully built oandapyV20


In [6]:
from config import access_token, accountID
def get_candles(n):
    #access_token='XXXXXXX'#you need token here generated from OANDA account
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


True
True
True


In [12]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    # EXECUTING ORDERS
    #accountID = "XXXXXXX" #your account ID here
    client = API(access_token)
         
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [ ]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()